# 2. 호모그래피를 이용한 이미지 매칭  

호모그래피 변환을 이용하여 두 영상 사이의 위치 및 자세 변환 관계를 파악할 수 있다. 
<br>

<img src="https://mblogthumb-phinf.pstatic.net/20151118_225/tlqor123_1447835351090RdDVu_PNG/epipolar_geometry2.png?type=w2" width="300" height="300">
<br>

## 2.1 호모그래피의 정의
---

두 카메라가 한 물체를 바라보고 있을 때, 두 카메라에 맺히는 이미지를 비교해 두 카메라 사이의 거리와 회전 관계를 알 수 있다. 이때 평면형 물체를 대상으로 할 경우, 두 카메라 사이의 변환행렬을 '호모그래피'라 한다. 아래는 호모그래피 $H$의 일반식이다. <br>

$$ x_2 = sHx_1 $$

$s$는 scale로, 스칼라 값을 가진다. 위의 식을 풀어쓰면 다음과 같다. <br><br>

$$ \begin{bmatrix}x_{2}\\y_{2}\\1 \end{bmatrix} = s
\begin{bmatrix}h _{11}&h _{12}&h _{12}\\h _{21}&h _{22}&h _{23}\\h _{31}&h _{32}&h _{33}\end{bmatrix}
\begin{bmatrix}x _{1}\\y _{1}\\1\end{bmatrix} $$
  
<br><br>

## 2.2 호모그래피 계산
---

호모그래피의 마지막 요소는 1로 고정되기 때문에 8개의 미지수를 구하면 호모그래피를 특정할 수 있다. 따라서, 매칭점 set $(x_1, y_1) , (x_2, y_2)$ 가 총 4set로 8개의 미지수를 구하여 호모그래피를 산출할 수 있다.<br><br>


### 2.2.1 DLT(Direct Linear Transform): 제차 방정식으로의 변형

해당 문제를 풀기 위해서는 8개의 연립방정식 계산이 필요하기 때문에 식을 단순화 할 필요가 있다. 호모그래피 일반식을 다음과 같은 과정을 거쳐 제차 형식으로 정리한다. <br><br>

$$ x_2 = (h _{11}x_1 + h _{12}y_1 + h _{13})/(h _{31}x_1 + h _{32}y_1 + h _{33}) $$
$$ y_2 = (h _{21}x_1 + h _{22}y_1 + h _{23})/(h _{31}x_1 + h _{32}y_1 + h _{33}) $$
<br><br>
$$ (h _{11}x_1 + h _{12}y_1 + h _{13}) - x_2(h _{31}x_1 + h _{32}y_1 + h _{33}) = 0 $$
$$ (h _{21}x_1 + h _{22}y_1 + h _{23}) - x_2(h _{31}x_1 + h _{32}y_1 + h _{33}) = 0 $$
<br><br>
$$ \begin{bmatrix}x_1&y_1&1&0&0&0&x_1x_2&y_1x_2&x_2\\
                  0&0&0&x_1&y_1&1&x_1y_2&y_1y_2&y_2 \end{bmatrix}
   \begin{bmatrix}h _{11}\\h _{12}\\h _{13}\\h _{21}\\h _{22}\\h _{23}\\h _{33}\\
   h _{31}\\h _{32}\\h _{33}\end{bmatrix}=0 $$
   <br><br>
위의 연립방정식을 단순하게 다음과 같이 나타낼 수 있다.
<br><br>
$$ \begin{bmatrix}a_{x}\\ a_{y}\end{bmatrix}h=0 $$
<br><br>
   
매칭점 1개의 set에 대해 위와 같이 2개의 선형 제차 연립방정식 형태로 정리할 수 있으며, 매칭점 4개의 set에 대해 동일하게 적용하면 다음과 같이 8개의 선형 제차 연립방정식의 형태로 나타낼 수 있다.

$$ \begin{bmatrix}a_{x1}\\ a_{y1}\\ a_{x2}\\ a_{y2}\\ a_{x3}\\ a_{y3}\\ a_{x4}\\ a_{y4} \end{bmatrix}h=0 $$
<br><br>

왼쪽 행렬은 $(8\times 9)$ 크기의 행렬 $A$로 단순하게 표현하면 다음과 같이 나타낼 수 있다.

$$Ah=0$$


해당 문제는 $Ah = 0$을 푸는 문제로 변환되어, SVD를 응용하여 위의 제차방정식을 풀어낼 수 있다. <br><br>


### 2.2.2 SVD(Singular Value Decomposition)의 응용

SVD는 행렬 분해 방법 중 하나로 정사각 행렬을 대상으로 하는 고윳값 분해를 $m\times  n$ 행렬로 일반화시킨 방법이다. 이를 이용하여 $m\times n$크기의 행렬 $A$를 다음 3가지 행렬로 분해할 수 있다.
<br><br>

$$ A = U\Sigma V^T $$

* $U$(left singular vector): $AA^T$의 고유벡터를 열벡터로 하는 행렬. $(m\times m)$
* $\Sigma$: $A$의 특이값을 대각원소로 하는 행렬. $(m\times n)$
* $V$(right singular vector): $A^TA$의 고유벡터를 열벡터로 하는 행렬. $(n\times n)$
<br><br>


SVD에는 더 많은 내용이 내포되어있지만 우리는 여기서 행렬들의 크기와 $V$의 마지막 열벡터에만 집중하면 된다.

위의 식의 양변에 $V$를 곱해 우리는 다음과 같이 나타낼 수 있다. 
<br>

$$ AV = U\Sigma $$
<br>

이때 $A$는 $(8\times 9)$ 행렬로 8개의 특이값을 가지기 때문에 특이값 행렬 $\Sigma$의 마지막 열벡터는 영벡터로 구성되는 것을 알 수 있다. 이러한 특징을 이용하여 $ Ah=0 $을 풀어낸다.

위의 식의 양변에 $(9\times 1)$크기의 열벡터 $ \begin{bmatrix} 0 & 0 & \cdots & 1\end{bmatrix}^T $를 곱하면 $V$와 $\Sigma$는 마지막 열벡터만 남게 된다. $V$의 마지막 열벡터는 $v_9$, $\Sigma$의 마지막 열벡터는 영벡터이며 다음과 같이 식을 정리할 수 있다.

$$ Av_9=0 $$

즉, 우리는 $V$의 마지막 열벡터를 호모그래피 요소 열벡터 $h(9\times 1)$로 사용할 수 있다. 호모그래피의 마지막 요소는 1로 고정되기 때문에 $h$를 $(3\times 3)$행렬로 변환한 뒤 $h_{33}$으로 나눠주어 최종 호모그래피 행렬 $H$를 구할 수 있다.



### 2.2.3 Python 구현

In [7]:
import cv2 
import numpy as np

# 매칭점 set 초기화
pts1 = [[1,1], [2,1], [1,2], [2,2]]
pts2 = [[1,2], [2,3], [3,4], [2,5]]

# DLT 계산 (matrix A 계산)
index_list = [1, 2, 3, 4]
A = np.empty((0,9))
for n in index_list:
    A = np.append(A,np.array([[pts1[n,0,0], pts1[n,0,1], 1, 0, 0, 0, -pts1[n,0,0]*pts2[n,0,0], -pts1[n,0,1]*pts2[n,0,0], -pts2[n,0,0]]]),axis=0)
    A = np.append(A,np.array([[0, 0, 0, pts1[n,0,0], pts1[n,0,1], 1, -pts1[n,0,0]*pts2[n,0,1], -pts1[n,0,1]*pts2[n,0,1], -pts2[n,0,1]]]),axis=0)

# 특이값 분해(SVD)
[U,D,Vh] = np.linalg.svd(A, full_matrices=True)

# 호모그래피 H 산출
h = Vh[-1]
s = h[-1]
H = h.reshape(3,3)/s

ModuleNotFoundError: No module named 'cv2'

---
작성자: 김윤중  
e-mail: yunjung.kim181@gmail.com  
github: YJ_Kim (@Muun-Muun)